In [ ]:
import pandas as pd
import seaborn as sns
import time
%pylab inline

In [ ]:
import scipy.stats as stats
import statsmodels.formula.api as sm

In [ ]:
for package in ['pandas', 'seaborn', 'scipy', 'statsmodels']:
    print(sys.modules[package].__name__, sys.modules[package].__version__)

---
## Load samples metadata

In [ ]:
# TARA metadata
samples_metadata_file = "../data/TARA_metadata.csv"
metadata_df = pd.read_csv(samples_metadata_file)

In [ ]:
metadata_df_slim = metadata_df[["dataset", "Depth", "Protocol_Label", "Region", "Station", "fraction"]].copy()

---
## Load genes metadata

In [ ]:
gene_annotation = pd.read_csv('../data/gene_info.tsv', sep='\t')

In [ ]:
gene_annotation_slim = gene_annotation[['vsearch_hash', 'seq_type', 'tax', 'gene']]

In [ ]:
# Keep viral and marine cyanobacteria sequences
gene_annotation_slim = gene_annotation_slim[gene_annotation_slim['tax'].isin(['MIC', 'VIR'])]

---
## Load salmon results

In [ ]:
df_rpkm = pd.read_csv('../data/salmon_tximport_edger_rpkm.tsv', sep='\t')

---

In [ ]:
def df_massage(df, gene_meta_df=gene_annotation_slim, samples_meta=metadata_df_slim):
    tpm = df.reset_index().rename(columns={'index':'vsearch_hash'})
    
    # df -> tidy
    raw_df = tpm.melt(id_vars='vsearch_hash', var_name='dataset', value_name='rpkm')
    
    # merge gene metadata
    tpm = tpm.merge(gene_meta_df, on='vsearch_hash', how='left')
    raw_df = raw_df.merge(gene_meta_df, on='vsearch_hash', how='left')

    raw_df = raw_df.merge(samples_meta[['dataset', 'Region', 'fraction', 'Station', 'Depth']], on='dataset', how='left')
         
    tpm_T_df = tpm.groupby(['seq_type']).agg(sum)
    tpm_T_df = tpm_T_df.T
    
    genes_fams = list()
    
    for i in tpm_T_df.sum().index: # display total number of mapped reads per "sequence type", except for petB
        if 'petB' not in i:
            genes_fams.append(i)
        #    print(i, '\t',tpm_T_df.sum().loc[i])
        if i == 'desC_fam1':
            vfad1_sum = tpm_T_df.sum().loc[i]
        elif i == 'desC_fam2':
            vfad2_sum = tpm_T_df.sum().loc[i]
    
    print("\n")
    print("ratio vfad1/vfad2: {}".format(vfad1_sum/vfad2_sum))
    
    summary_df = tpm_T_df[genes_fams].describe()

    tpm_T_df['dataset'] = tpm_T_df.index
    tpm_T_df = tpm_T_df.merge(samples_meta[['dataset', 'Region', 'fraction', 'Station', 'Depth']], on='dataset', how='left')
    tpm_T_df.set_index('dataset', inplace=True)

    return summary_df, raw_df, tpm_T_df

In [ ]:
sumdf, raw_df,dfm = df_massage(df_rpkm)

> #paper Fam1 represents the bulk of desC recruited reads, with up to 46 times the recruited fam2 reads.

---
## DesC stats

In [ ]:
# group by station/depth
dfm_loc = dfm.groupby(['Station', 'Depth']).agg(sum).reset_index()

In [ ]:
dfm_loc['myo'] = (dfm_loc['gp20_Myoviridae'] + dfm_loc['gp23_Myoviridae'])/2

In [ ]:
for fam in ['desC_fam1', 'desC_fam2']:
    dfm_loc[fam + '_ratio'] = dfm_loc[fam] / dfm_loc['myo']

In [ ]:
dfm_loc.sort_values(by='desC_fam1_ratio', ascending=False)['desC_fam1_ratio'].head()

In [ ]:
dfm_loc.loc[dfm_loc['desC_fam1'] > 0]['desC_fam1_ratio'].describe()

In [ ]:
dfm_loc.loc[((dfm_loc['desC_fam1'] > 0) & (dfm_loc['desC_fam1_ratio'] < 1))]['desC_fam1_ratio'].describe()

> #paper In the samples where desC_fam1 was present, was present in up to 305% of total cyanomyophages with an average 7% of cyanomyophages carrying the gene.

In [ ]:
dfm_loc.loc[dfm_loc['desC_fam2_ratio'] > 0][['desC_fam2_ratio']].describe()

> #paper In the samples where desC_fam2 was present, was present in up to 3.5% of total cyanomyophages with an average 0.1% of cyanomyophages carrying the gene.

In [ ]:
fam2_v_fam1_df = dfm_loc.loc[(dfm_loc['desC_fam2_ratio'] > 0) & (dfm_loc['desC_fam1_ratio'] > 0)][['Station', 'Depth', 'desC_fam1', 'desC_fam2']].copy()

In [ ]:
fam2_v_fam1_df['fam2/totalDesC'] = fam2_v_fam1_df['desC_fam2'] / (fam2_v_fam1_df['desC_fam1']+fam2_v_fam1_df['desC_fam2'])

In [ ]:
fam2_v_fam1_df.describe()

> #paper In samples where both desC families were detected, fam2 desaturases accounts in average for 16% of the total viral DesC and up to 43%.

---
# plotting parameters

In [ ]:
sns.set_style("whitegrid")

---
## DesC geo origin

In [ ]:
total_desC_fam1 = dfm['desC_fam1'].sum()
total_desC_fam2 = dfm['desC_fam2'].sum()

In [ ]:
desC_df = dfm[['desC_fam1', 'desC_fam2', 'Region']].copy()
desC_df['fam1_ratio'] = (desC_df['desC_fam1']/total_desC_fam1) * 100
desC_df['fam2_ratio'] = (desC_df['desC_fam2']/total_desC_fam2) * 100

In [ ]:
desC_df = desC_df[['fam1_ratio', 'fam2_ratio', 'Region']].copy()

desC_df = desC_df.melt(id_vars=['Region'], value_name='counts', var_name='fam')
desC_df = desC_df.loc[desC_df['counts'] > 0]

desC_df = desC_df.groupby(['fam','Region']).agg(sum).reset_index()


In [ ]:
desC_df.sort_values(by=['fam','counts'], ascending=False)

> #paper Fam1 recruited reads originated from all the sampled regions except for the SO, with ~82% of the total recruitments  coming from two regions the IO (47%) and SPO (35.7%). In contrast, for fam2 ~98% of the total recruitments came from SPO(66%) and SAO(32%) with the remaining coming from the IO.

#### empty boxplot with individual points on top

In [ ]:
plot_df_ = dfm[['desC_fam1', 'desC_fam2', 'Region']].copy()

plot_df_ = plot_df_.melt(id_vars=['Region'], value_name='counts', var_name='fam')
plot_df_ = plot_df_.loc[plot_df_['counts'] > 0]

# add missing regions for desC_fam2
plot_df_ =plot_df_.append(pd.DataFrame([['RS', 'desC_fam2', 0],\
              ['MS', 'desC_fam2', 0], ['NAO', 'desC_fam2', 0],\
              ['NPO', 'desC_fam2', 0]], columns=plot_df_.columns), ignore_index=True)

# column order

column_order = plot_df_.loc[plot_df_['fam']=='desC_fam1'][['Region', 'counts']].groupby('Region')\
.agg(pd.np.median).sort_values(by='counts', ascending=False).index.tolist()


# set plot parameters
color_dict = {'desC_fam2':"#7137c8", 'desC_fam1':"#aa8800"}
#fam1=
sns.set_context("paper", font_scale=2)
sns.set_style('ticks')
sns.despine()

# fig initialization
g = sns.FacetGrid(plot_df_, col="fam", sharey=True, sharex=False, size=6,aspect=1)

# add box plot
(g.map(sns.boxplot, "Region", "counts", "fam", palette=color_dict, order=column_order))

# remove boxplot fill and set colors based on 
# https://stackoverflow.com/questions/36874697/how-to-edit-properties-of-whiskers-fliers-caps-etc-in-seaborn-boxplot
for subp in [0,1]:
    for i, artist in enumerate(g.fig.get_axes()[subp].artists):
        col = artist.get_facecolor()
        if subp == 1:
            col = 'white'
        artist.set_edgecolor(col)
        artist.set_facecolor('white')

        for j in range(i*6,i*6+6):
            line = g.fig.get_axes()[subp].lines[j]
            line.set_color(col)
            line.set_mfc(col)
            line.set_mec(col)

# Add in points to show each observation
(g.map(sns.swarmplot, "Region", "counts", "fam", order=column_order, dodge=True, palette=color_dict, size=4))

g.set_axis_labels("Region", "RPKM").set_titles("{col_name}")

g.fig.get_axes()[0].set_yscale('log')

g.savefig("./box_plot_v1_b.pdf")
plt.show()



> #paper Box plot of viral desaturases family 1 and family 2 relative abundance grouped by oceanic provinces. Each box represents the different sampling stations combined.

> Units: Reads per Kilobase per Million

> The data for the 178 metagenomes with presence of viral desaturases is shown in a box plot with median, 25th percentile, 75th percentile, minimum and maximum values depicted.

### per fraction

In [ ]:
desC_df = dfm[['desC_fam1', 'desC_fam2', 'Region', 'fraction']].copy()

desC_df = desC_df.melt(id_vars=['fraction', 'Region'], value_name='counts', var_name='fam')
desC_df = desC_df.loc[desC_df['fam'].isin(['desC_fam1', 'desC_fam2'])]
desC_df = desC_df.loc[desC_df['counts'] > 1]

fraction_df = desC_df.groupby(['fraction', 'fam']).agg(sum).reset_index().sort_values(by='fam')


In [ ]:
total_fam1 = fraction_df.loc[fraction_df['fam'] == 'desC_fam1']['counts'].sum()
total_fam2 = fraction_df.loc[fraction_df['fam'] == 'desC_fam2']['counts'].sum()

In [ ]:
for fam in ['desC_fam1', 'desC_fam2']:
    total = fraction_df.loc[fraction_df['fam'] == fam]['counts'].sum()
    fraction_df.loc[fraction_df['fam'] == fam, '%'] = fraction_df['counts']/total*100

In [ ]:
fraction_df

> #paper 71.01% of the fam1 recruited reads originated from the bacterial fraction, 18.98% from the giant virus fraction and the remaining 10.01% from the viral fraction

> #paper fam2 recruited reads came from: 79.09% giant virus fraction, 18.35% viral fraction and, 2.54% from the bacterial fraction

In [ ]:
plot_df_ = dfm[['desC_fam1', 'desC_fam2', 'Region', 'fraction']].copy()

plot_df_ = plot_df_.melt(id_vars=['fraction', 'Region'], value_name='counts', var_name='fam')
plot_df_ = plot_df_.loc[plot_df_['fam'].isin(['desC_fam1', 'desC_fam2'])]
plot_df_ = plot_df_.loc[plot_df_['counts'] > 1]

color_dict = {'BACT':'black', 'VIRUS':'r', 'GIRUS':'b'}

# column order

column_order = plot_df_.loc[plot_df_['fam']=='desC_fam1'][['Region', 'counts']].groupby('Region')\
.agg(pd.np.median).sort_values(by='counts', ascending=False).index.tolist()

# fig initialization
sns.set_context("paper", font_scale=2)

g = sns.FacetGrid(plot_df_, row="fam", sharey=True, sharex=False, size=6,aspect=3)

# add box plot
(g.map(sns.boxplot, "Region", "counts", "fraction", palette=color_dict, order=column_order))


# remove boxplot fill and set colors based on 
# https://stackoverflow.com/questions/36874697/how-to-edit-properties-of-whiskers-fliers-caps-etc-in-seaborn-boxplot
for subp in [0,1]:
    for i, artist in enumerate(g.fig.get_axes()[subp].artists):
        col = artist.get_facecolor()
        if subp == 1:
            col = 'white'
        artist.set_edgecolor(col)
        artist.set_facecolor('white')

        for j in range(i*6,i*6+6):
            line = g.fig.get_axes()[subp].lines[j]
            line.set_color(col)
            line.set_mfc(col)
            line.set_mec(col)

# Add in points to show each observation
(g.map(sns.swarmplot, "Region", "counts", "fraction", order=column_order, dodge=True, palette=color_dict, size=6))

# set log scale
g.fig.get_axes()[0].set_yscale('log')
g.fig.get_axes()[1].set_yscale('log')

g.set_axis_labels("Region", "RPKM").set_titles("{row_name} desC relative abundance")

g.add_legend()

plt.show()

g.savefig('./reads_distribution_per_fraction.pdf')

---
# correlations


---

## desC v petB per region

In [ ]:
# creates a dictionary containing the correlation dataframes (pearson coeff and P value) per region
dic_df = dict()
fams = ['desC_fam1', 'desC_fam2']
corr_list_ = list()
for fam in fams:
    dic_df[fam] = dict()
    fam_df_ = dfm.loc[dfm[fam] > 0]
    for gn, g in fam_df_.groupby('Region'):
        dic_df[fam][gn] = dict()
        
        g2 = g[g.columns.tolist()[:-4]]

        pearson_list_ = list()
        header = ['fam', 'seq_type', 'r', 'r2', 'p', 'nz', 'n', 'region']

        for seq_type in g2.columns:
            if seq_type not in fams:
                if 'petB' in seq_type:
                    pearson = stats.pearsonr(g2[fam], g2[seq_type])

                    nz_ = int(g2[seq_type].astype(bool).sum(axis=0))

                    row = [fam, seq_type.replace('|', '_'), pearson[0], pearson[0]*pearson[0], pearson[1], nz_,len(g2), gn]

                    pearson_list_.append(row)
        
        g2_corr_df = pd.DataFrame(pearson_list_, columns=header)
        g2_corr_df = g2_corr_df.sort_values(by=['r2', 'nz'], ascending=[0,0])
        
        corr_list_.append(g2_corr_df)
        
        dic_df[fam][gn]['df'] = g2.rename(columns=lambda x: x.replace('|', '_'))
        print('fam:{}, group: {}; # of samples: {}'.format(fam, gn, len(g2)))
corr_df = pd.concat(corr_list_).sort_values(by='r2', ascending=False)
corr_df = corr_df.dropna()

In [ ]:
dic_df.keys()

In [ ]:
dic_df['desC_fam1'].keys()

In [ ]:
f = 'desC_fam1'
reg = 'NAO'
st = 'petB_Pro_LLI_LLIA'
df_ = dic_df[f][reg]['df'][[f, st]]


In [ ]:
df_.plot(st, f, kind='scatter')

## Detect and remove outliers

In [ ]:
def adjust(DF,x_,y_,plot_filename, axis_labels):
    print("x:{}".format(x_))
    print("y:{}".format(y_))

    formula_ = x_ + ' ~ ' + y_
    df__ = DF.reset_index()

    model_ = sm.ols(formula=formula_, data=df_)
    res_ = model_.fit()

    print("r2={}, r2_adj={}".format(res_.rsquared, res_.rsquared_adj))
    print("p={}".format(res_.f_pvalue))
    
    # outliers identification
    influence = res_.get_influence()
    (c, p) = influence.cooks_distance
    df__['C'] = c

    # show datasets to remove

    outliers_num = len(df__[df__['C'] > 1])
    outliers_id = df__[df__['C'] > 1]['dataset'].values.tolist()

    print("Identified {0} outiers: {1}".format(outliers_num, ','.join(outliers_id)))

    reduced_df_ = df__[df__['C'] < 1].copy()

    # fit again

    formula_ = x_ + ' ~ ' + y_
    model_ = sm.ols(formula=formula_, data=reduced_df_)
    res_ = model_.fit()
    print("After removal of outlier\n r2={}, r2_adj={}".format(res_.rsquared, res_.rsquared_adj))
    print("p={}".format(res_.f_pvalue))
    slope = float(res_.params.loc[y_])
    intercept = float(res_.params.loc["Intercept"])
    m_ = 1/slope
    c_ = -1 * intercept/slope
    if c_ < 0 :
        c_ = c_ * -1
        equation = r"$y=${}$x-${}".format(round(m_,4), round(c_,4))
    else:
        equation = r"$y=${}$x+${}".format(round(m_,4), round(c_,4))
    print("Equation\n{}".format(equation))

    # plot with reduced
    color_dict = {'desC_fam2':"#7137c8", 'desC_fam1':"#aa8800"}
    
    g = sns.lmplot(x_, y_, data=reduced_df_, ci=None, aspect=1.6,\
           scatter_kws={"color": color_dict[y_]},\
           line_kws={'color': 'black', 'alpha':0.5})
    
    g.set_axis_labels(axis_labels[0], axis_labels[1])
    plt.annotate(equation, xy=(30000,5000), xytext=(30000, 5000))
    plt.annotate(r'$R^2$ = {}'.format(round(res_.rsquared_adj, 4)), xy=(30000,2000), xytext=(30000, 2000))
    g.savefig(plot_filename)
    return reduced_df_


In [ ]:
df_

In [ ]:
st

In [ ]:
f

In [ ]:
red_df = adjust(df_, st, f, './fam1-ProLLIA_correlation.pdf', (r"$\mathit{petB}$ Pro LLIA", r"$\mathit{desC}$ vFAD-I"))

> #paper Abundances of viral desC family 1 and Prochlorococcus LLI-A were significantly correlated (P < 0.001; R2=0.91; the regression line, regression equation and R 2 value are shown)
> #paper the sample ERR599095 was determined to be an outlier based on Cooks distance > 1 and removed from the plot.

---
# TSV file generation for map

In [ ]:
map_metadata_columns = ['Station','Longitude_Start', 'Latitude_Start']

In [ ]:
map_df = dfm[['desC_fam1', 'desC_fam2', 'Station', 'Region']]

In [ ]:
map_df = map_df.groupby(['Station', 'Region']).agg(sum).reset_index()

In [ ]:
map_df = map_df.melt(value_vars=['desC_fam1', 'desC_fam2'], id_vars=['Station', 'Region'],\
                     var_name='family', value_name='rpkm')

In [ ]:
map_df = map_df.merge(metadata_df[map_metadata_columns], on='Station', how='left')

In [ ]:
map_df = map_df.rename(columns=lambda x: x.replace('_Start',''))

In [ ]:
map_df = map_df.drop_duplicates(['Station', 'Region', 'family', 'rpkm'])

In [ ]:
map_df.to_csv('../data/map_df.tsv', sep='\t', index=False)